In [2]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 7.0MB/s 
     |████████████████████████████████| 1.5MB 7.3MB/s 
     |████████████████████████████████| 194kB 8.7MB/s 
     |████████████████████████████████| 235kB 16.8MB/s 
     |████████████████████████████████| 2.2MB 16.5MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 


In [114]:
import torch
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx

data_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/0.txt'
ans_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/0_score.txt'

ans = {}
with open(ans_path) as f:
  line = f.readline()
  while line:
    key, value = line.split()
    key = eval(key)
    ans[key] = eval(value)
    line = f.readline()

G = nx.read_edgelist(data_path, nodetype = int)
edge_index = from_networkx(G).edge_index
x = torch.tensor([[G.degree(node),1,1] for node in G.nodes], dtype=torch.float)
y = torch.tensor([ans[node] for node in G.nodes], dtype=torch.float)
data = Data(edge_index=edge_index, x=x, y=y)
data

Data(edge_index=[2, 39964], x=[5000, 3], y=[5000])

In [4]:
def g(x):
  return 1 / (1+1/torch.exp(x))

class customLoss(torch.nn.Module):
  def __init__(self):
    super(customLoss, self).__init__()
    

  def forward(self, predict, y):
    yij = torch.stack([g(predict[i] - predict[j]) for i, j in G.edges])
    bij = torch.tensor([g(y[i] - y[j]) for i, j in G.edges])
    loss = -bij * torch.log(yij) - (1 - bij) * torch.log(1 - yij)
    loss = torch.sum(loss)
    return loss

In [70]:
import torch
from torch.nn import Linear, GRUCell, ReLU
from torch_geometric.nn import GCNConv
from torch.nn import Sequential as Seq

class Net(torch.nn.Module):
  def __init__(self, L):
    super(Net, self).__init__()
    self.L = L-1 # encoder layer number
    self.data_in = Seq(Linear(data.num_features, 4), ReLU())
    self.Aggregation = GCNConv(4, 4)
    self.Combine = GRUCell(4, 4)
    self.data_out = Seq(Linear(4, 5), ReLU(), Linear(5,1))

  def forward(self, x, edge_index):
    # encoder
    h = self.data_in(x)
    h = h / torch.norm(h) # dimension
    layer = torch.tensor([])
    layer = torch.cat((layer, torch.unsqueeze(h, 0)))
    for _ in range(self.L):
      hn = self.Aggregation(h, edge_index)
      h = self.Combine(h, hn)
      h = h / torch.norm(h)
      layer = torch.cat((layer, torch.unsqueeze(h, 0)))
    z, _ = torch.max(layer, 0)

    # decoder
    y = self.data_out(z)
    y = torch.squeeze(y)
    return y

In [71]:
model = Net(5)
criterion = customLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(110):
  optimizer.zero_grad()  # Clear gradients.  
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  # print(out)
  loss = criterion(out, data.y)  # Compute the loss.
  print(f'Epoch {epoch}: {loss}')
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  top_n_accuracy(out, 1)

tensor([0.7183, 0.7113, 0.7152,  ..., 0.6800, 0.6800, 0.6800],
       grad_fn=<SqueezeBackward0>)
Epoch 0: 13850.060546875
tensor([0.7125, 0.7047, 0.7091,  ..., 0.6696, 0.6696, 0.6696],
       grad_fn=<SqueezeBackward0>)
Epoch 1: 13850.0283203125
tensor([0.7242, 0.7155, 0.7204,  ..., 0.6766, 0.6766, 0.6766],
       grad_fn=<SqueezeBackward0>)
Epoch 2: 13849.9990234375
tensor([0.7380, 0.7284, 0.7337,  ..., 0.6855, 0.6855, 0.6855],
       grad_fn=<SqueezeBackward0>)
Epoch 3: 13849.9697265625
tensor([0.7525, 0.7420, 0.7478,  ..., 0.6950, 0.6950, 0.6950],
       grad_fn=<SqueezeBackward0>)
Epoch 4: 13849.9453125
tensor([0.7629, 0.7515, 0.7579,  ..., 0.7002, 0.7002, 0.7002],
       grad_fn=<SqueezeBackward0>)
Epoch 5: 13849.92578125
tensor([0.7694, 0.7570, 0.7639,  ..., 0.7014, 0.7014, 0.7014],
       grad_fn=<SqueezeBackward0>)
Epoch 6: 13849.908203125
tensor([0.7752, 0.7617, 0.7692,  ..., 0.7017, 0.7017, 0.7017],
       grad_fn=<SqueezeBackward0>)
Epoch 7: 13849.896484375
tensor([0.7806, 

In [120]:
def top_n_accuracy(train, n, ans):
  num = int(len(train) * n/100)
  train = train.tolist()
  predict = {}
  for i in range(len(train)):
    predict[i] = train[i]

  predict = sorted(predict.items(), key = lambda x: x[1], reverse = True)[:num]
  ans = sorted(ans.items(), key = lambda x: x[1], reverse = True)[:num]
  hit = 0
  for i in range(num):
    if predict[i][0] == ans[i][0]:
      hit += 1
  print(predict)
  print(ans)
top_n_accuracy(out, 1, ans)

[(0, 0.6973639726638794), (2, 0.6886624097824097), (1, 0.6771048307418823), (240, 0.6680325269699097), (411, 0.6540981531143188), (412, 0.6540771722793579), (242, 0.6501765251159668), (544, 0.6405585408210754), (244, 0.638791561126709), (4, 0.632236659526825), (610, 0.6313621997833252), (9, 0.6282909512519836), (772, 0.6281698942184448), (543, 0.6272611618041992), (773, 0.6270633339881897), (8, 0.6259691119194031), (771, 0.6238293647766113), (419, 0.6235950589179993), (5, 0.6229208707809448), (17, 0.6226246356964111), (10, 0.6222177743911743), (546, 0.6217952370643616), (413, 0.6215094923973083), (1675, 0.6214934587478638), (1115, 0.6213051080703735), (3, 0.620647668838501), (243, 0.6202442049980164), (1550, 0.6195394396781921), (616, 0.6189274191856384), (775, 0.6182246208190918), (611, 0.6177647113800049), (1676, 0.6168626546859741), (19, 0.6161899566650391), (7, 0.6154876947402954), (246, 0.6146746873855591), (417, 0.6142374873161316), (414, 0.6135773658752441), (26, 0.6133463978767